In [ ]:
# | default_exp create_vegetation_parameters

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import os
import warnings
import numpy as np
from pathlib import Path
from typing import Dict, List
from sureau_ecos_py.soil_utils import (
    compute_theta_at_given_p_soil,
    compute_theta_at_given_p_soil_camp,
)
from sureau_ecos_py.plant_utils import read_vegetation_file
from sureau_ecos_py.plant_utils import convert_f_cm3_to_v_mm
from sureau_ecos_py.create_soil_parameters import create_soil_parameters
from sureau_ecos_py.create_modeling_options import create_modeling_options
from sureau_ecos_py.create_stand_parameters import create_stand_parameters

In [ ]:
# | export

def create_vegetation_parameters(
    stand_parameters:Dict, # Dictionary created using the `create_stand_parameters` function
    soil_parameters:Dict, # Dictionary created using the `create_soil_parameters` function
    file_path: Path = None,  # Path to a csv file containing lai_max, latitude and longitude values i.e path/to/parameter_values.csv
    list_of_parameters: List = None,  # A list containing the necessary input parameters instead of reading them in file. Will only be used if 'file_path' arguement is not provided
    modeling_options: Dict = None,  # Dictionary created using the `create_modeling_options` function
    sep: str = ";",  # CSV file separator can be ',' or ';'


) -> Dict:  # Dictionary containing parameters to run the model

    "Create a dictionary with the vegetation parameters to run \code{run.SurEauR}"

    # Assert parameters ---------------------------------------------------------

    # Make sure the file_path exist or is None
    assert file_path is None or os.path.exists(
        file_path
    ), f"Path: {file_path} not found, check spelling or set file_path = None"


    # Raise error if file_path and list_of_parameters both are missing
    if file_path is None and list_of_parameters is None:
        raise ValueError(
            "Both file_path and list_of_parameters are missing, provide one of them"
        )

    # Raise error if file_path and list_of_parameters both are provided
    if file_path is not None and list_of_parameters is not None:
        raise ValueError(
            "Both file_path and list_of_parameters are provided, only one of these two arguments should be used"
        )

    # Delete as soon the option is implemented
    if list_of_parameters is not None:
        raise ValueError(
            "Option for including list_of_parameters not implemented yet"
        )


    # Make sure that modeling_options is a dictionary
    assert isinstance(
        modeling_options, Dict
    ), f"modeling_options must be a dictionary not a {type(modeling_options)}"

    # Make sure that stand_parameters is a dictionary
    assert isinstance(
        stand_parameters, Dict
    ), f"stand_parameters must be a dictionary not a {type(stand_parameters)}"

    # Make sure that soil_parameters is a dictionary
    assert isinstance(
        soil_parameters, Dict
    ), f"soil_parameters must be a dictionary not a {type(soil_parameters)}"


    # Create vegetation_parameters from function inputs -------------------------

    # Read CSV file
    vegetation_parameters = read_vegetation_file(file_path = file_path,
                                                 modeling_options = modeling_options,
                                                 sep = sep)

    # Compute stomatal response parameters for the sigmoid from P12_gs and P88_gs
    # provided in the CSV file
    if modeling_options["stomatal_reg_formulation"] == "sigmoid":

        # Calculate p50_gs
        if "p50_gs" not in vegetation_parameters:
            vegetation_parameters["p50_gs"] = (vegetation_parameters["p88_gs"] + vegetation_parameters["p12_gs"])/2
            vegetation_parameters["slope_gs"] = 100/(vegetation_parameters["p12_gs"] - vegetation_parameters["p88_gs"])


    # Get maximum leaf area index of the stand (LAImax) from stand parameters
    vegetation_parameters["lai_max"] = stand_parameters["lai_max"]

    # Calculate root distribution within each soil layer (Jackson et al. 1996)
    warnings.warn("Make sure that depth from soil_parameters is in meters")

    # Create numpy array
    vegetation_parameters["root_distribution"] = np.array([0, 0, 0], dtype=float)

    # soil_parameters["depth"][n] is multiplied by 100 to convert it from meters
    # to centimeters

    # Layer 1
    vegetation_parameters["root_distribution"][0] = 1 - vegetation_parameters["betarootprofile"]**(soil_parameters["soil_depths"][0]*100)

    # Layer 2
    vegetation_parameters["root_distribution"][1] = (1 - vegetation_parameters["betarootprofile"]**(soil_parameters["soil_depths"][1]*100)) - vegetation_parameters["root_distribution"][0]

    # Layer 3
    vegetation_parameters["root_distribution"][2] = 1 - (vegetation_parameters["root_distribution"][0] + vegetation_parameters["root_distribution"][1])

    # Calculate turgor loss point (tlp)
    tlp = (vegetation_parameters["pifullturgor_leaf"]*vegetation_parameters["epsilonsym_leaf"])/vegetation_parameters["pifullturgor_leaf"] + vegetation_parameters["epsilonsym_leaf"]

    # Pedotransfer function == vg
    if modeling_options['pedo_transfer_formulation'] == "vg":

        theta_at_tlp = compute_theta_at_given_p_soil(psi_target= np.abs(tlp),
                                                     theta_res = soil_parameters['residual_capacity_vg'],
                                                     theta_sat = soil_parameters['saturation_capacity_vg'],
                                                     alpha_vg = soil_parameters['alpha_vg'],
                                                     n_vg = soil_parameters['n_vg']
                                                     )

        theta_at_p50 = compute_theta_at_given_p_soil(psi_target= np.abs(vegetation_parameters['p50_vc_leaf']),
                                                     theta_res = soil_parameters['residual_capacity_vg'],
                                                     theta_sat = soil_parameters['saturation_capacity_vg'],
                                                     alpha_vg = soil_parameters['alpha_vg'],
                                                     n_vg = soil_parameters['n_vg']
                                                     )

        # Compute TAW @Tlp @P12 & @P50 (Diagnoostic)
        vegetation_parameters['taw_at_tlp'] = np.sum(soil_parameters['v_field_capacity'] - convert_f_cm3_to_v_mm(x = theta_at_tlp,
                                                                                                                 rock_fragment_content = soil_parameters['rock_fragment_content'],
                                                                                                                 layer_thickness = soil_parameters['layer_thickness']
                                                                                                                 )
                                                     )

        vegetation_parameters['taw_at_p50'] = np.sum(soil_parameters['v_field_capacity'] - convert_f_cm3_to_v_mm(x = theta_at_p50,
                                                                                                                 rock_fragment_content = soil_parameters['rock_fragment_content'],
                                                                                                                 layer_thickness = soil_parameters['layer_thickness']
                                                                                                                 )
                                                     )


        print(f"Available water capacity @Tlp (VG):{vegetation_parameters['taw_at_tlp']} mm")
        print(f"Available water capacity @P50 (VG):{vegetation_parameters['taw_at_p50']} mm")

    # Pedotransfer function == Campbell
    elif modeling_options['pedo_transfer_formulation'] == "campbell":
        print("compute_theta_at_given_p_soil_camp")




    #if modeling_options['']
    return vegetation_parameters

## Create vegetation parameters 

In [ ]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="sigmoid",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="vg",
)
modeling_options_dict

defaultdict(list,
            {'constant_climate': False,
             'etp_formulation': 'pt',
             'rn_formulation': 'linear',
             'pedo_transfer_formulation': 'vg',
             'time_step_for_evapo': 2,
             'time': array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22]),
             'reset_swc': True,
             'avoid_water_soil_transfer': True,
             'comp_options': defaultdict(list,
                         {'numerical_scheme': 'implicit',
                          'nsmalltimesteps': 12.0,
                          'lsym': 1,
                          'ssym': 1,
                          'clapo': 1,
                          'ctapo': 1,
                          'eord': 1,
                          'lcav': 1,
                          'scav': 1}),
             'stomatal_reg_formulation': 'sigmoid',
             'soil_evapo': True,
             'defoliation': True,
             'threshold_mortality': 51,
             'transpiration_model': 'jar

In [ ]:
stand_params = create_stand_parameters(
    file_path="./sample_data/stand_example.csv",
    lai_max=None,
    latitude=None,
    longitude=None,
    sep=";",
)
stand_params

defaultdict(list, {'lai_max': 4.5, 'latitude': 48.73, 'longitude': 6.23})

In [ ]:
soil_params = create_soil_parameters(
    file_path="./sample_data/soil_example.csv",
    modeling_options = modeling_options_dict,
    default_soil=False,
    offset_psoil=1,
    psoil_at_field_capacity=1,
)
soil_params

There is an offset on Psoil of 1 MPa
Psoil at field capacity = 0.001 MPa
You are using vg pedotransfer formulation
Available water capacity Wilting: 25.65230545095943 mm
Available water capacity Residual: 135.33945663142987 mm
Can soil_params["v_soil_storage_capacity"] be negative?? Ask


defaultdict(list,
            {'offset_psoil': 1,
             'psoil_at_field_capacity': 0.001,
             'pedo_transfer_formulation': 'vg',
             'soil_depths': array([0.2, 1. , 4. ]),
             'layer_thickness': array([0.2, 0.8, 3. ]),
             'g_soil_0': 30.0,
             'rock_fragment_content': array([75., 82., 94.]),
             'alpha_vg': array([0.0005, 0.0005, 0.0005]),
             'n_vg': array([1.55, 1.55, 1.55]),
             'i_vg': array([0.5, 0.5, 0.5]),
             'ksat_vg': array([10., 10., 10.]),
             'saturation_capacity_vg': array([1., 1., 1.]),
             'residual_capacity_vg': array([0.098, 0.098, 0.098]),
             'wilting_point': 0.39128115288895826,
             'field_capacity': 0.45987020489687114,
             'v_field_capacity': array([22.99351024, 66.22130951, 82.77663688]),
             'v_saturation_capacity_vg': array([ 50., 144., 180.]),
             'v_residual_capacity_vg': array([ 4.9  , 14.112, 17.64 ]),
    

In [ ]:
create_vegetation_parameters(
    file_path="./sample_data/vegetation_example_wide.csv",
    list_of_parameters = None,
    soil_parameters = soil_params,
    stand_parameters = stand_params,
    modeling_options = modeling_options_dict,
    sep = ",",
)

frac_leaf_sym' set to 0.4
Available water capacity @Tlp (VG):108.5505012570013 mm
Available water capacity @P50 (VG):64.63695676128677 mm


/workspaces/sureau_ecos_py/sureau_ecos_py/plant_utils.py:805: UserWarning: Foliage evergreen has no params
  warnings.warn("Foliage evergreen has no params")
/tmp/ipykernel_493/692492318.py:78: UserWarning: Make sure that depth in from soil_parameters is in meters
  warnings.warn("Make sure that depth in from soil_parameters is in meters")


defaultdict(list,
            {'c_lapoinit': 1e-05,
             'c_sapoinit': 2e-05,
             'fcrit': 450,
             'foliage': 'evergreen',
             'jarvis_par': 0.006,
             'k': 0.5,
             'k_plantinit': 0.62,
             'k_ssyminit': 0.26,
             'ldmc': 570,
             'lma': 106,
             'p12_gs': -2.07,
             'p50_vc_leaf': -3.4,
             'p50_vc_stem': -3.4,
             'p88_gs': -2.62,
             'pt_coeff': 1.14,
             'pifullturgor_leaf': -2.1,
             'pifullturgor_stem': -2.1,
             'psi_close': -2,
             'psi_start_closing': -0.5,
             'q10_1_gmin': 1.2,
             'q10_2_gmin': 4.8,
             'tphase_gmin': 37.5,
             'tbase': 3,
             'tgs_optim': 25,
             'tgs_sens': 17,
             'apofrac_leaf': 0.4,
             'apofrac_stem': 0.4,
             'betarootprofile': 0.97,
             'canopystorageparam': 1.5,
             'daystart': 55,
         